<font size=6> <b> Advanced Python : week #5</b> </font>
<div class="alert alert-block alert-success">
   Advanced Python features <br>
    <ol>
        <li> asyncio : iterator/generator/coroutine </li>
    </ol>
</div>

<p style="text-align:right;"> sumyeon@gmail.com </p>

# Asyncio : Iterator/Generator/Coroutine
- coroutine is a core concept of asynchronous programming which is provided by 'asyncio' module
- coroutine is implemented using the syntax/mechanism of generator
- generator is a kind of iterator 

## iterable/iterator

### list, dict, set, tuple are iterable objects

In [1]:
a = [2,4,6,8,10]

In [2]:
for val in a:
    print(val)

2
4
6
8
10


### mechanism under iteration

- iterable object is that can return its members one by one
 > technically, iterable objects are those with __iter__ method which will return iterator
- iterator is an object which can be iterated one by one
 > technically, iterator are those with __next__ method which will return its members one by one. 

In [3]:
a = [2,4,6,8,10]
'__iter__' in a.__class__.__dict__

True

In [4]:
iter_a =  iter(a)
print(type(iter_a))

<class 'list_iterator'>


In [5]:
'__next__' in iter_a.__class__.__dict__

True

In [6]:
next(iter_a), next(iter_a), next(iter_a), next(iter_a), next(iter_a)

(2, 4, 6, 8, 10)

In [7]:
next(iter_a)

StopIteration: 

### example of an iterable class

In [8]:
class SimpleRange:
    """Class to implement an iterator
    of powers of two"""

    def __init__(self, max=0):
        self.max = max

    def __iter__(self):
        self.n = 0
        return self

    def __next__(self):
        if self.n < self.max:
            result = self.n
            self.n += 1
            return result
        else:
            raise StopIteration

In [9]:
# for - in syntax first call the iter function, and call the next function till StopIteration exception
for i in SimpleRange(5):
    print(i)

0
1
2
3
4


## Generator
- A kind of iterator, but simpler than iterator at coding
- technicall, a generator object is the one with 'yield'
- 'yield' is a keyword that is used like return, except the function will return a generator

### An Example

In [10]:
def create_generator():
    for i in range(4):
        yield i*i

In [11]:
gen_instance = create_generator()

print(gen_instance)

<generator object create_generator at 0x000001D2FABF16C8>


In [12]:
print(next(gen_instance))

print(next(gen_instance))

print(next(gen_instance))

print(next(gen_instance))

0
1
4
9


In [13]:
print(next(gen_instance))

StopIteration: 

In [14]:
for i in create_generator():
    print(i)

0
1
4
9


### Comparison with Iterator
- the instance creation will match the '__iter__' call of Iterator
- code before the 'yield' and the 'yield' itself will match the '__next__' calls
- at the end of 'yield', the unmatched '__next__' call will raise StopIteration exception

In [15]:
def gen_template():
    print("before 1st yield")
    yield 1
    print("after 1st bet before the 2nd yield")
    yield 2
    print("after 2nd yield")

In [16]:
next(gen_template())

before 1st yield


1

In [17]:
gen_iter = iter(gen_template())

In [18]:
next(gen_iter)

before 1st yield


1

In [19]:
next(gen_iter)

after 1st bet before the 2nd yield


2

In [20]:
next(gen_iter)

after 2nd yield


StopIteration: 

### yield from
- syntatic sugar for the below frequently used pattern <br>

<pre>
    for xxx in source: 
        yield xxx
=>
    yield from source
</pre>

In [21]:
def baz():
    for i in range(5):
        yield i

def bar():
    for i in range(5,10):
        yield i

def foo():
    for v in baz():
        yield v
    for v in bar():
        yield v

for v in foo():
    print(v)

0
1
2
3
4
5
6
7
8
9


In [22]:
def baz():
    for i in range(5):
        yield i

def bar():
    for i in range(5,10):
        yield i

def foo():
    yield from baz()
    yield from bar()

for v in foo():
    print(v)

0
1
2
3
4
5
6
7
8
9


## Coroutine
- is a building block of asynchronous programming
- functions whose execution can be paused/suspended at a particular point
- while generators are data producuers, coroutines are data consumers

> feedback = (yield xxxx) template get the feedback from the caller and save it into feedback <br>
> the caller can send back the feedback to generaor using 'send' <br>
> NOTE: "next(generator)" is exactly same as "generator.send(None)" <br>

### Simple Coroutine
> coroutine which is alomost like generator but using feedback

In [23]:
def foo():
    msg = yield  # coroutine feature
    yield msg    # generator feature

coro = foo()

next(coro)
result = coro.send("bar")
print(result) 

bar


### Another Simple Coroutine : Stop-control

In [24]:
import random
def magic_pot(start=1, end=1000):
    while True:
        stop = (yield random.randint(start, end))
        print("stop %s" % stop)
        if stop is True:
            yield "magic pot stopped"
            break

In [25]:
gen = magic_pot()

In [26]:
print(gen)

<generator object magic_pot at 0x000001D2FABF17C8>


In [27]:
print(next(gen))

287


In [28]:
print("second")

second


In [29]:
print(gen.send(False))  # input value for generator

stop False
153


In [30]:
print(gen.send(True))  # input value for generator

stop True
magic pot stopped


In [31]:
try:
    print(next(gen))
except StopIteration:
    print('iteration stopped')

iteration stopped


### Another Simple Coroutine : Status Control
- prime number generator, but you can set the starting number at any time

In [32]:
# normal python method which check whether the give value is prime or not
def isPrime(n):
    if n < 2 or n % 1 > 0:
        return False
    elif n == 2 or n == 3:
        return True
    for x in range(2, int(n**0.5) + 1):
        if n % x == 0:
            return False
    return True

In [33]:
# prime generator
def getPrimes():
    value = 0
    while True:
        if isPrime(value):
            i = yield value
            if i is not None:
                value = i
        value += 1

In [34]:
primes = getPrimes()

In [35]:
next(primes)

2

In [36]:
next(primes)

3

In [37]:
primes.send(100)

101

In [38]:
next(primes)

103

In [39]:
primes.send(300)

307

In [40]:
next(primes)

311

In [41]:
primes.close()

In [42]:
primes.throw(ValueError, "Too Large")

ValueError: Too Large

## Generator Coroutine
> coroutine which use '@asyncio.coroutine' decorator with 'yield from'

In [43]:
import asyncio
import random

@asyncio.coroutine
def compute_coroutine(x):
    yield from asyncio.sleep(random.random()) # yield from native coroutine
    print(x * 2)

# asyncio.run(compute_coroutine(2))
await compute_coroutine(2)


4


### (Native) Coroutine
- async/await instead of '@asynio.coroutin'/'yield from'

In [44]:
import asyncio

async def f1():
    print('before')
    await asyncio.sleep(1)
    print('after 1 sec')
    

In [45]:
# asyncio.run(f1()) 
await f1()

before
after 1 sec


## Coroutine Event Loop Simplified (from ver 3.7)

### Before 3.7

In [46]:
import asyncio


async def foo():
    print('Running in foo')
    await asyncio.sleep(0)
    print('Explicit context switch to foo again')


async def bar():
    print('Explicit context to bar')
    await asyncio.sleep(0)
    print('Implicit context switch back to bar')


ioloop = asyncio.get_event_loop()
tasks = [ioloop.create_task(foo()), ioloop.create_task(bar())]
wait_tasks = asyncio.wait(tasks)

#ioloop.run_until_complete(wait_tasks)
#ioloop.close()

Running in foo
Explicit context to bar
Explicit context switch to foo again
Implicit context switch back to bar


In [47]:
print(tasks[0].result())

None


In [48]:
import asyncio


async def foo():
    print('Running in foo')
    await asyncio.sleep(0)
    print('Explicit context switch to foo again')


async def bar():
    print('Explicit context to bar')
    await asyncio.sleep(0)
    print('Implicit context switch back to bar')


async def main():
    tasks = [foo(), bar()]
    return await asyncio.gather(*tasks)


#asyncio.run(main())
await main()

Running in foo
Explicit context to bar
Explicit context switch to foo again
Implicit context switch back to bar


[None, None]

# Generator/Coroutine Summary

<div class="alert alert-block alert-info">
    <b>Generator</b> <br>
    - yield <br>
    - yield from (ver 3.3)
    <blockquote> bridge between caller & returning generator </blockquote> 
    <br>
    <b> Coroutine</b>
    <blockquote> similar to generator, but not just a producer but also data consumer</blockquote>
    - asyncio (ver3.4)
    <blockquote> coroutine was implmented using generator syntax <br>
        ex) @asyncio.coroutine, yield from </blockquote> <br>
    - 'async def'/'await' (ver 3.5)
    <blockquote> new keyword 'async','await','@types.coroutine' are introduced <br>
        async def = @types.coroutine = @asyncio.coroutine <br>
        await = yield from </blockquote> <br>
    - asyncio simpler syntax introduced (ver 3.7)
    <blockquote> boiler-plate code for task creation & event loop handler are simplified </blockquote>
</div>

# [Reference] coroutine examples

### synchronous example

In [49]:
import time

def sync_task_1():
    print('sync_task_1 start')
    print('sync_task_1 sleeping for 3 secs')
    time.sleep(3)
    print('sync_task_1 complete')
    
def sync_task_2():
    print('sync_task_2 start')
    print('sync_task_2 sleeping for 2 secs')
    time.sleep(2)
    print('sync_task_2 complete')
    
start = time.time()
sync_task_1()
sync_task_2()
end = time.time()
print(end-start)

sync_task_1 start
sync_task_1 sleeping for 3 secs
sync_task_1 complete
sync_task_2 start
sync_task_2 sleeping for 2 secs
sync_task_2 complete
5.013146877288818


### asynchronous example - wrong

In [50]:
import asyncio
import time

async def sync_task_1():
    print('sync_task_1 start')
    print('sync_task_1 sleeping for 3 secs')
    await asyncio.sleep(3)
    print('sync_task_1 complete')
    
async def sync_task_2():
    print('sync_task_2 start')
    print('sync_task_2 sleeping for 2 secs')
    await asyncio.sleep(2)
    print('sync_task_2 complete')
    
async def main():
    start = time.time()
    await sync_task_1()
    await sync_task_2()
    end = time.time()
    print(end-start)
    
#asyncio.run(main())
await main()

sync_task_1 start
sync_task_1 sleeping for 3 secs
sync_task_1 complete
sync_task_2 start
sync_task_2 sleeping for 2 secs
sync_task_2 complete
5.01580810546875


### asynchronous example - correct

In [51]:
import asyncio
import time

async def sync_task_1():
    print('sync_task_1 start')
    print('sync_task_1 sleeping for 3 secs')
    await asyncio.sleep(3)
    print('sync_task_1 complete')
    
async def sync_task_2():
    print('sync_task_2 start')
    print('sync_task_2 sleeping for 2 secs')
    await asyncio.sleep(2)
    print('sync_task_2 complete')
    
async def main():
    start = time.time()
    task1 =  asyncio.create_task(sync_task_1())
    task2 =  asyncio.create_task(sync_task_2())
    await task1
    await task2
    end = time.time()
    print(end-start)
    
#asyncio.run(main())
await main()

sync_task_1 start
sync_task_1 sleeping for 3 secs
sync_task_2 start
sync_task_2 sleeping for 2 secs
sync_task_2 complete
sync_task_1 complete
3.0097382068634033


### asynchronous example - correct & simple

In [52]:
import asyncio
import time

async def sync_task_1():
    print('sync_task_1 start')
    print('sync_task_1 sleeping for 3 secs')
    await asyncio.sleep(3)
    print('sync_task_1 complete')
    
async def sync_task_2():
    print('sync_task_2 start')
    print('sync_task_2 sleeping for 2 secs')
    await asyncio.sleep(2)
    print('sync_task_2 complete')
    
async def main():
    start = time.time()
    await asyncio.gather(sync_task_1(), sync_task_2())
    print(end-start)
    
#asyncio.run(main())
await main()

sync_task_1 start
sync_task_1 sleeping for 3 secs
sync_task_2 start
sync_task_2 sleeping for 2 secs
sync_task_2 complete
sync_task_1 complete
-8.093514680862427


### asynchronous example - correct & simple & returning value

In [53]:
import asyncio

async def factorial(name, number):
    f = 1
    if number < 0:
        raise ValueError('Exception')
    for i in range(2, number+1):
        print(f"Task {name}: Compute factoral({i})...")
        await asyncio.sleep(1)
        f *= i
    print(f"Task {name}: factorial({number}) = {f}")
    return f

In [54]:
async def main_normal():
    result = await asyncio.gather(factorial("A",2), factorial("B",3), factorial("C",4))
    print(result)

#asyncio.run(main_normal())
await main_normal()

Task A: Compute factoral(2)...
Task B: Compute factoral(2)...
Task C: Compute factoral(2)...
Task A: factorial(2) = 2
Task B: Compute factoral(3)...
Task C: Compute factoral(3)...
Task B: factorial(3) = 6
Task C: Compute factoral(4)...
Task C: factorial(4) = 24
[2, 6, 24]


In [55]:
async def main_broken():
    result = await asyncio.gather(factorial("A",2), factorial("B",3), factorial("C",-1))
    print(result)
    
#asyncio.run(main_broken())
await main_broken()

Task A: Compute factoral(2)...
Task B: Compute factoral(2)...


ValueError: Exception

In [ ]:
async def main_resilent():
    result = await asyncio.gather(factorial("A",2), factorial("B",3), factorial("C",-1), return_exceptions=True)
    print(result)
    
#asyncio.run(main_resilent())
await main_resilent()

Task B: Compute factoral(3)...
Task A: factorial(2) = 2
Task B: factorial(3) = 6
